In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fl_g13.config import RAW_DATA_DIR
from torchvision import datasets, transforms

from fl_g13.modeling import train
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

2025-04-17 11:36:17.092 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


### Load data

In [3]:
transform = transforms.Compose([
    transforms.ToTensor()
])
cifar100_train = datasets.CIFAR100(root=RAW_DATA_DIR, train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root=RAW_DATA_DIR, train=False, download=True, transform=transform)

### Train and save model

In [4]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))     # -> [B, 16, 32, 32]
        x = F.max_pool2d(x, 2)        # -> [B, 16, 16, 16]
        x = F.relu(self.conv2(x))     # -> [B, 32, 16, 16]
        x = F.max_pool2d(x, 2)        # -> [B, 32, 8, 8]
        x = x.view(x.size(0), -1)     # -> [B, 32*8*8]
        x = self.fc1(x)               # -> [B, 100]
        return x

In [5]:
checkpoint_dir = "/home/massimiliano/Projects/fl-g13/checkpoints"

# Parameters
batch_size  = 32
start_epoch = 1
num_epochs  = 2
save_every  = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataloader = torch.utils.data.DataLoader(cifar100_train, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=True)

model = TinyCNN(100)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.04)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
train(
    checkpoint_dir=checkpoint_dir,
    prefix="", # Will automatically generate a name for the model
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    criterion=criterion,
    start_epoch=start_epoch,
    num_epochs=num_epochs,
    save_every=save_every,
    model=model,
    optimizer=optimizer,
    scheduler=None,
    verbose=False,
)

No prefix/name for the model was provided, choosen prefix/name: funky_pidgeot_86
🚀 Epoch [1/2] Completed (50.00)
	📊 Training Loss: 4.1296
	✅ Training Accuracy: 8.55%
🔍 Validation Results:
	📉 Validation Loss: 3.8385
	🎯 Validation Accuracy: 13.70%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/funky_pidgeot_86_epoch_1.pth
🚀 Epoch [2/2] Completed (100.00)
	📊 Training Loss: 3.6752
	✅ Training Accuracy: 16.41%
🔍 Validation Results:
	📉 Validation Loss: 3.5947
	🎯 Validation Accuracy: 17.53%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/funky_pidgeot_86_epoch_2.pth


In [7]:
train(
    checkpoint_dir=checkpoint_dir,
    prefix="TinyCNN", # Setting a name for the model
    start_epoch=start_epoch,
    num_epochs=num_epochs,
    save_every=save_every,
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    model=model, # Use the same model as before (partially pre-trained)
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,
    verbose=False,
)

🚀 Epoch [1/2] Completed (50.00)
	📊 Training Loss: 3.4709
	✅ Training Accuracy: 19.97%
🔍 Validation Results:
	📉 Validation Loss: 3.4428
	🎯 Validation Accuracy: 20.44%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/TinyCNN_epoch_1.pth
🚀 Epoch [2/2] Completed (100.00)
	📊 Training Loss: 3.3281
	✅ Training Accuracy: 22.50%
🔍 Validation Results:
	📉 Validation Loss: 3.3345
	🎯 Validation Accuracy: 22.65%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/TinyCNN_epoch_2.pth


**Resume training**

In [8]:
from fl_g13.modeling import load

# Generate untrained objects
model2 = TinyCNN(num_classes=100)
optimizer2 = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.04)
criterion2 = torch.nn.CrossEntropyLoss()

# Load the model from the latest checkpoint
path = checkpoint_dir + "/TinyCNN_epoch_2.pth"
start_epoch = load(path=path, model=model2, optimizer=optimizer2, scheduler=None)

✅ Loaded checkpoint from /home/massimiliano/Projects/fl-g13/checkpoints/TinyCNN_epoch_2.pth, resuming at epoch 3


In [9]:
num_epochs = 4
save_every = 2

train(
    checkpoint_dir=checkpoint_dir,
    prefix="TinyCNN", # Use the same name as before to continue training!
    start_epoch=start_epoch, # Now start epoch is not 1 (will resume from where it was left)
    num_epochs=num_epochs, # This is not the number of epochs to reach, but how many to do starting from now!
    save_every=save_every,
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    model=model2,
    criterion=criterion2,
    optimizer=optimizer2,
    scheduler=None,
    verbose=False,
)

🚀 Epoch [1/4] Completed (25.00)
	📊 Training Loss: 3.2481
	✅ Training Accuracy: 23.98%
🔍 Validation Results:
	📉 Validation Loss: 3.3341
	🎯 Validation Accuracy: 22.65%
🚀 Epoch [2/4] Completed (50.00)
	📊 Training Loss: 3.2483
	✅ Training Accuracy: 23.98%
🔍 Validation Results:
	📉 Validation Loss: 3.3358
	🎯 Validation Accuracy: 22.65%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/TinyCNN_epoch_4.pth
🚀 Epoch [3/4] Completed (75.00)
	📊 Training Loss: 3.2482
	✅ Training Accuracy: 23.98%
🔍 Validation Results:
	📉 Validation Loss: 3.3346
	🎯 Validation Accuracy: 22.65%
🚀 Epoch [4/4] Completed (100.00)
	📊 Training Loss: 3.2481
	✅ Training Accuracy: 23.98%
🔍 Validation Results:
	📉 Validation Loss: 3.3348
	🎯 Validation Accuracy: 22.65%
💾 Saved checkpoint at: /home/massimiliano/Projects/fl-g13/checkpoints/TinyCNN_epoch_6.pth
